# Imports & Funcoes

In [285]:
import requests
import datetime as dt
from datetime import date, datetime
import numpy as np
import pandas as pd
from dateutil.relativedelta import relativedelta

In [286]:
import fdb


class db():
    conConfServer = {
        'database':'Pc-server:c:/Sacwin/Dados/Sacwin21.fdb', 
        'user':'SYSDBA', 
        'password':'masterkey'
    }
    conConfLocal = {
        'database':'F:/Users/felip/Desktop/Millenius/Data/backup/Sacwin21.fdb', 
        'user':'SYSDBA',
        'password':'masterkey'
    }
    conn = {}
    
    def connect(self):
        #self.conn = fdb.connect(database = 'F:/Users/felip/Desktop/Millenius/Data/backup/Sacwin21.fdb', user = 'SYSDBA', password = 'masterkey')
        self.conn = fdb.connect(database = 'Pc-server:c:/Sacwin/Dados/Sacwin21.fdb', user = 'SYSDBA', password = 'masterkey')
        return self.conn.cursor()
    
    def fetchOneSQL(self, sql):
        sql = sql.upper()
        cursor = self.connect()
        cursor.execute(sql.upper())
        result = cursor.fetchone()
        return result
    
    def fetchAllSQL(self, sql):
        cursor = self.connect()
        cursor.execute(sql.upper())
        result = cursor.fetchall()
        return result
    
datab = db()

In [287]:
class conversorSemanas:
    def __init__(self, anoSemana) -> None:
        self.dataConvertida = self.converterData(anoSemana)
        self.mesConvertido = self.dataConvertida.strftime("%m/%Y")
        self.anoConvertido = self.dataConvertida.year
        
    def converterData(self, anoSemana):
        ano, semana = map(int, anoSemana.split('/'))
        if ano < 100:
            # Convertendo ano abreviado para ano de 4 dígitos
            ano = 2000 + ano if ano <= dt.datetime.now().year % 100 else 1900 + ano
        
        # Definir o primeiro dia da semana
        data_primeiro_dia_semana = dt.datetime.strptime(f'{ano}-W{semana-1}-1', "%Y-W%W-%w")
        
        # Ajustar para segunda-feira (0) em vez de domingo (6)
        if data_primeiro_dia_semana.weekday() == 0:
            data_primeiro_dia_semana += dt.timedelta(weeks=1)
        
        data_completa = data_primeiro_dia_semana.date()
        return pd.to_datetime(data_completa).normalize()
    
# conversor = conversorSemanas('23/02')

# Dados

## Estrutural

In [288]:
### Estrutura Base ###
"""
tabela = ''
dados = datab.fetchAllSQL(f'SELECT * FROM {tabela}')
indexItens = datab.fetchAllSQL(f"select rdb$field_name from rdb$relation_fields where rdb$relation_name='{tabela}'")
# indexItens = []
dados = pd.DataFrame(dados)
dados.columns = (indexItens)
"""
### Tratar anos/sem cobrança
"""
    .apply(lambda x: pd.to_datetime(x+'/1', format= '%y/%W/%w' ))
"""



"\n    .apply(lambda x: pd.to_datetime(x+'/1', format= '%y/%W/%w' ))\n"

In [289]:
tabelas = datab.fetchAllSQL('''SELECT a.RDB$RELATION_NAME
                            FROM RDB$RELATIONS a
                            WHERE COALESCE(RDB$SYSTEM_FLAG, 0) = 0 AND RDB$RELATION_TYPE = 0''')
df = pd.DataFrame({'Tabelas': tabelas})
df.to_csv('data/lista.txt', sep='\t', index=False, header=False)

## Artigos

In [290]:
tabela = 'GRUPOSARTIGOS'
grupos = datab.fetchAllSQL(f'SELECT * FROM {tabela}')
indexItens = datab.fetchAllSQL(f"select rdb$field_name from rdb$relation_fields where rdb$relation_name='{tabela}'")
indexItens = ["id_grupo", "descricao_grupo", "valor"]
grupos = pd.DataFrame(grupos)
grupos.columns = (indexItens)
grupos.to_csv('data/grupos.csv', sep= ';', index= False)

In [291]:
tabela = 'ARTIGOSPRECOS'
precos = datab.fetchAllSQL(f'SELECT * FROM {tabela}')
#indexItens = datab.fetchAllSQL(f"select rdb$field_name from rdb$relation_fields where rdb$relation_name='{tabela}'")
indexItens = ['ref', 'tabela', 'preco_artigo', 'porcentagem_desconto','valor_desconto', 'valor_sugerido', 'comissao_fixa', 'valor_acerto_vendedor']
precos = pd.DataFrame(precos)

precos.columns = (indexItens)
precos = precos.query('tabela == 1')

In [292]:
tabela = 'ARTIGOS'
artigos = datab.fetchAllSQL(f'SELECT * FROM {tabela}')
# indexItens = datab.fetchAllSQL(f"select rdb$field_name from rdb$relation_fields where rdb$relation_name='{tabela}'")
indexItens = ['ref', 'artigo', 'custo', 'etiqueta', 'id_grupo', 'fornecedor', 'status_artigo', 'nome_modelo', 'ano_modelo', 'ncm', 'grade', 'comissao', 'obs', 'kit', 'caixa', 'atualizar' ]
artigos = pd.DataFrame(artigos)
artigos.columns = (indexItens)

In [293]:
artigos = artigos.drop(columns= ['custo', 'nome_modelo','etiqueta', 'fornecedor', 'ano_modelo', 'ncm', 'grade', 'comissao', 'kit', 'caixa', 'atualizar'])  # type: ignore
artigos = pd.merge(artigos, grupos, on= 'id_grupo')
artigos.name = 'artigos'
artigos = pd.merge(artigos, precos[['ref', 'preco_artigo']], on='ref')

In [294]:
artigos.head()

,ref,artigo,id_grupo,status_artigo,obs,descricao_grupo,valor,preco_artigo
0,10006,TANGA SEM COSTURA DIVERSAS,11.0,ATIVO,,TANGA COMUM,0,18.9
1,10007,TANGA COTTON DIVERSAS,11.0,ATIVO,,TANGA COMUM,0,12.9
2,10009,TANGA ROMANTIC DIVERSAS,11.0,ATIVO,,TANGA COMUM,0,12.6
3,10016,TANGA POLI,11.0,ATIVO,,TANGA COMUM,0,15.9
4,10017,TANGA MODA,11.0,ATIVO,,TANGA COMUM,0,13.9


In [295]:
artigos.to_csv('data/artigos.csv', sep= ';', index= False)

## Estoque Artigos

In [296]:
tabela = 'ARTIGOSCENTROESTOQUE'
dados = datab.fetchAllSQL(f'SELECT * FROM {tabela}')
indexItens = datab.fetchAllSQL(f"select rdb$field_name from rdb$relation_fields where rdb$relation_name='{tabela}'")
indexItens = ['ref', 'id_centro_estoque', 'tamanho', 'codigo_de_barras', 'estoque_deposito', 'estoque_rua', 'estoque_embalagem', 'estoque_total', 'quantidade_caixa']
dados = pd.DataFrame(dados)
dados.columns = (indexItens)

In [297]:
dados = dados.drop(columns= ['id_centro_estoque', 'codigo_de_barras', 'estoque_rua', 'estoque_embalagem', 'estoque_total', 'quantidade_caixa'])


In [298]:
dados.to_csv('data/estoque.csv', sep= ';', index= False)

## Promotor

In [299]:
tabela = 'VENDEDORES'
vendedores = datab.fetchAllSQL(f'SELECT * FROM {tabela}')
indexItens = datab.fetchAllSQL(f"select rdb$field_name from rdb$relation_fields where rdb$relation_name='{tabela}'")

In [300]:
tabela = 'VENDEDORES'
vendedores = datab.fetchAllSQL(f'SELECT * FROM {tabela}')
indexItens = datab.fetchAllSQL(f"select rdb$field_name from rdb$relation_fields where rdb$relation_name='{tabela}'")
indexItens = ['id_vendedor', 'descricao_vendedor', 'status_vendedor', 'admissao', 'comissao', 'endereco', 'complemento', 'bairro', 'cidade', 'estado', 'cep', 'telefone', 'rg', 'cpf', 'nascimento', 'placa', 'email', 'sincroniza_auto', 'ordem_artigo_tablet', 'ordens_artigo_novo_tablet', 'editar_comissao', 'zerar_fec', 'conituar_fec', 'impressao_auto', 'alterar_fec', 'sugerir_artigo', 'inicio_cobranca', 'imprimir_pedido_novo', 'exportar_clientes_sacd', 'deixar_pepino', 'fazer_pedido_novo_rua', 'sugerir_cobranca', 'mensagem_debito', 'exportar_digitados', 'exportar_pepinos', 'devolver_tudo', 'fec_nao_entregue', 'artigo_por_comissao', 'somente_leitor', 'inativa_cli_tablet', 'cpf_obrigatorio', 'rec_pepino_fec', 'percentual_kit', 'bloquear_cliente', 'verificar_saldo', 'tabela_precovenda_sacdroid','excluir_venda','edita_comissao_venda', 'utilizar_garantias','devolver_dinheiro_garantias','alterar_aniversario','gps_obrigatorio', 'exibir_financeiro', 'exibir_devolucao', 'bloquear_apos_entregue', 'quantidade_editavel', 'comissao_carteira', 'valor_meta', 'abaixo_meta', 'acima_meta', 'pepinos', 'acertos', 'adicionar_acertos', 'pagar_pepino', 'sincronizar_24h', 'id_comissao', 'comissao_art_sem_comissao', 'comissao_fixa', 'utilizar_notas_reais', 'quantidade_limite_de_credito', 'multiplicador_de_credito', 'valor_limite_abertura', 'pedido_novo', 'valor_pedido_novo', 'valor_pedido_reativar', 'utilizar_vendas', 'tipo_pix', 'chave_pix']
vendedores = pd.DataFrame(vendedores)
vendedores.columns = (indexItens)

In [301]:
vendedores = vendedores.drop(columns= ['email', 'sincroniza_auto', 'ordem_artigo_tablet', 'ordens_artigo_novo_tablet', 'editar_comissao', 'zerar_fec', 'conituar_fec', 'impressao_auto', 'alterar_fec', 'sugerir_artigo', 'inicio_cobranca', 'imprimir_pedido_novo', 'exportar_clientes_sacd', 'deixar_pepino', 'fazer_pedido_novo_rua', 'sugerir_cobranca', 'mensagem_debito', 'exportar_digitados', 'exportar_pepinos', 'devolver_tudo', 'fec_nao_entregue', 'artigo_por_comissao', 'somente_leitor', 'inativa_cli_tablet', 'cpf_obrigatorio', 'rec_pepino_fec', 'percentual_kit', 'bloquear_cliente', 'verificar_saldo', 'tabela_precovenda_sacdroid','excluir_venda','edita_comissao_venda', 'utilizar_garantias','devolver_dinheiro_garantias','alterar_aniversario','gps_obrigatorio', 'exibir_financeiro', 'exibir_devolucao', 'bloquear_apos_entregue', 'quantidade_editavel', 'comissao_carteira', 'valor_meta', 'abaixo_meta', 'acima_meta', 'pepinos', 'acertos', 'adicionar_acertos', 'pagar_pepino', 'sincronizar_24h', 'id_comissao', 'comissao_art_sem_comissao', 'comissao_fixa', 'utilizar_notas_reais', 'quantidade_limite_de_credito', 'multiplicador_de_credito', 'valor_limite_abertura', 'pedido_novo', 'valor_pedido_novo', 'valor_pedido_reativar', 'utilizar_vendas', 'tipo_pix', 'chave_pix'])
vendedores.name = 'vendedores'

In [302]:
vendedores.to_csv('data/vendedores.csv', sep= ';', index= False)

## Praças

In [303]:
tabela = 'PRACAS'
pracas = datab.fetchAllSQL(f'SELECT * FROM {tabela}')
indexItens = datab.fetchAllSQL(f"select rdb$field_name from rdb$relation_fields where rdb$relation_name='{tabela}'")
indexItens = ['id_praca', 'descricao_praca', 'id_vendedor', 'estoque', 'tipo', 'id_tabela', 'percentual_media', 'semanas_cobranca', 'status_praca', 'limite_credito', 'usar_mensagem', 'mensagem_1', 'mensagem_2', 'mensagem_3', 'aceita_fora_carga', 'obs', 'comissao_fixa', 'multi_kit', 'coletor']
pracas = pd.DataFrame(pracas)
pracas.columns = (indexItens)

In [304]:
pracas = pracas.drop(columns= ['estoque', 'id_tabela', 'percentual_media', 'semanas_cobranca', 'limite_credito', 'usar_mensagem', 'mensagem_2', 'mensagem_3', 'aceita_fora_carga','comissao_fixa', 'multi_kit', 'coletor'])  # type: ignore


In [305]:
pracas = pracas.merge(vendedores[['id_vendedor','descricao_vendedor']], on= 'id_vendedor')

In [306]:
pracas.to_csv(f'data/pracas.csv', sep= ';', index= False)

## Sacoleiras

In [307]:
tabela = 'SACOLEIRAS'
sacoleiras = datab.fetchAllSQL(f'SELECT * FROM {tabela}')
indexItens = datab.fetchAllSQL(f"select rdb$field_name from rdb$relation_fields where rdb$relation_name='{tabela}'")
indexItens = ['id_praca', 'id_sacoleira', 'nome', 'status', 'primeiro_pedido', 'ultimo_pedido', 'comissao', 'indicacao', 'identidade', 'cpf', 'dia_mes', 'ano', 'endereco', 'complemento', 'bairro', 'cidade', 'uf', 'cep', 'telefone', 'email', 'observacao', 'ponto_antes', 'ponto_atual', 'ultimo_pedido_impresso', 'ano_semana_ultimo_pedido_impresso', 'id_contafinanceira', 'perfil', 'sacpro', 'valor_limite_credito', 'data_nascimento', 'contatos', 'gps', 'sacdroid', 'pai', 'mae', 'conjuge', 'spc', 'equipe', 'id_nivel', 'atualizar', 'classificacao', 'data_hora_inclusao', 'data_hora_reserva']
sacoleiras = pd.DataFrame(sacoleiras)
sacoleiras.columns = (indexItens)

In [308]:
infoCidades = pd.read_csv('data/cidades.csv', sep=';')

In [309]:
sacoleiras = sacoleiras.drop(columns= ['comissao', 'indicacao', 'dia_mes', 'ano', 'email', 'ponto_antes', 'ponto_atual', 'ultimo_pedido_impresso', 'ano_semana_ultimo_pedido_impresso', 'id_contafinanceira', 'perfil', 'sacpro', 'valor_limite_credito', 'contatos', 'sacdroid', 'pai', 'mae', 'conjuge', 'spc', 'equipe', 'id_nivel', 'atualizar', 'classificacao', 'data_hora_inclusao', 'data_hora_reserva'])

In [310]:
sacoleiras = sacoleiras.merge(infoCidades[['uf','cidade', 'latitude','longitude']], on=['cidade', 'uf'])

In [311]:
sacoleiras['idade'] = sacoleiras['data_nascimento'].apply(lambda x: 0 if x == None else (date.today().year - x.year))
sacoleiras.name = 'sacoleiras'

In [312]:
sacoleiras.to_csv(f'data/sacoleiras.csv', sep= ';', index= False)

## Cargas

In [313]:
tabela = 'CARGAS'
cargas = datab.fetchAllSQL(f'SELECT * FROM {tabela}')
# indexItens = datab.fetchAllSQL(f"select rdb$field_name from rdb$relation_fields where rdb$relation_name='{tabela}'")
indexItens = ['id_praca', 'ano_sem_pedido', 'ano_sem_cobranca', 'status', 'pedidos_digitados', 'pedidos_fechados', 'pedidos_pepinos', 'quantidade_total_recebida', 'valor_total_pedidos', 'quantidade_itens', 'valor_total_carga', 'pedidos_transferidos', 'data_lancamento', 'em_uso', 'qtd_kit', 'qtd_kit2', 'qtd_kit3', 'multikit', 'coletor', 'bloqueado']
cargas = pd.DataFrame(cargas)
cargas.columns = (indexItens)

In [314]:
cargas = cargas.drop(columns= ['pedidos_digitados', 'pedidos_fechados', 'pedidos_pepinos', 'quantidade_total_recebida', 'valor_total_pedidos', 'quantidade_itens', 'valor_total_carga', 'pedidos_transferidos', 'data_lancamento', 'em_uso', 'qtd_kit', 'qtd_kit2', 'qtd_kit3', 'multikit', 'coletor'])

In [315]:
cargas['ano_sem_pedido_data'] = cargas['ano_sem_pedido'].apply(lambda x: pd.to_datetime(x+'/1', format= '%y/%W/%w' ))
cargas['ano_sem_cobranca_data'] = cargas['ano_sem_cobranca'].apply(lambda x: pd.to_datetime(x+'/1', format= '%y/%W/%w' ))
cargas.name = 'cargas'

In [316]:
cargas.to_csv(f'data/cargas.csv', sep= ';', index= False)

## Cargas Artigos

In [317]:
tabela = 'CARGASARTIGOS'
cargaArtigos = datab.fetchAllSQL(f'SELECT * FROM {tabela}')
indexItens = datab.fetchAllSQL(f"select rdb$field_name from rdb$relation_fields where rdb$relation_name='{tabela}'")
indexItens = ['id_praca', 'ano_sem_pedido', 'id_artigo', 'id_centro_estoque', 'valor_unitario', 'valor_total', 'kit', 'quantidade_recebida', 'quantidade_distribuida', 'quantidade_devolvida', 'quantidade_brinde', 'quantidade_retorno', 'status', 'ordem', 'tabela', 'data_troca', 'id_artigo_old', 'quantidade_rua', 'quantidade_inserida', 'comissao_fixa', 'kit2', 'kit3', 'complemento', 'ordem2', 'ordem3', 'ordemc', 'valor_acerto_vendedor', 'comissao', 'quantidade_suja', 'quantidade_defeito']
cargaArtigos = pd.DataFrame(cargaArtigos)
cargaArtigos.columns = (indexItens)  # type: ignore

In [318]:
cargaArtigos = cargaArtigos.drop(columns=['id_centro_estoque', 'comissao_fixa','kit', 'kit2', 'kit3', 'complemento', 'ordem2', 'ordem3', 'ordemc', 'valor_acerto_vendedor', 'comissao', 'quantidade_suja', 'quantidade_defeito', 'ordem', 'tabela','id_artigo_old', 'quantidade_rua', 'quantidade_inserida', 'data_troca'])

In [319]:
## Tratamento de semanas
cargaArtigos['ano_sem_pedido_data'] = cargaArtigos['ano_sem_pedido'].apply(lambda x: pd.to_datetime(x+'/1', format= '%y/%W/%w' ))
cargaArtigos.name = 'cargaArtigos'


In [320]:
cargaArtigos.to_csv(f'data/cargaArtigos.csv', sep= ';', index= False)

## Pedidos

In [321]:
tabela = 'PEDIDOS'
pedidos = datab.fetchAllSQL(f'SELECT * FROM {tabela}')
indexItens = datab.fetchAllSQL(f"select rdb$field_name from rdb$relation_fields where rdb$relation_name='{tabela}'")
indexItens = ['id_pedido', 'ano_sem_pedido', 'ano_sem_cobranca', 'id_praca', 'id_sacoleira', 'status', 'quantidade_entregue', 'valor_total_sem_comissao', 'valor_total_com_comissao', 'valor_total_pedido', 'data_retorno', 'obs_encomenda', 'atualizacao', 'impresso', 'coletor', 'data_pedido', 'transferido', 'id_sacoleira_old', 'gps', 'gps_original', 'id_pedido_provisorio', 'id_mostruario', 'em_uso', 'digitador', 'valor_total_comissao_fixa', 'situacao', 'atualizar']
pedidos = pd.DataFrame(pedidos)
pedidos.columns = (indexItens) # type: ignore

In [322]:
pedidos = pedidos.drop(columns= ['obs_encomenda', 'atualizacao', 'impresso', 'coletor', 'transferido', 'id_sacoleira_old', 'gps', 'gps_original', 'id_mostruario', 'em_uso', 'digitador', 'valor_total_comissao_fixa', 'situacao', 'atualizar', 'id_pedido_provisorio'])

In [323]:
pedidos = pd.merge(pedidos, pracas[['id_vendedor', 'id_praca', 'descricao_praca', 'descricao_vendedor']], on= 'id_praca')
pedidos = pedidos.merge(sacoleiras[['id_sacoleira', 'cidade', 'uf', 'latitude', 'longitude']], on= 'id_sacoleira')

In [324]:
pedidos['status'].unique()

array(['FECHADO', 'PEPINO', 'DIGITADO', 'TRANSFERIDO'], dtype=object)

In [325]:
todosAnoSem = pd.concat([pedidos['ano_sem_pedido'], pedidos['ano_sem_cobranca']]).unique() # type: ignore

pedidos['ano_sem_pedido_data'] = pedidos['ano_sem_pedido']
pedidos['ano_mes_pedido'] = pedidos['ano_sem_pedido']
pedidos['ano_pedido'] = pedidos['ano_sem_pedido']

pedidos['ano_sem_cobranca_data'] = pedidos['ano_sem_cobranca']
pedidos['ano_mes_cobranca'] = pedidos['ano_sem_cobranca']
pedidos['ano_cobranca'] = pedidos['ano_sem_cobranca']

for anoSemana in todosAnoSem:
    conversor = conversorSemanas(anoSemana)
    pedidos['ano_sem_pedido_data'] = pedidos['ano_sem_pedido_data'].replace(anoSemana, conversor.dataConvertida)
    pedidos['ano_mes_pedido'] = pedidos['ano_mes_pedido'].replace(anoSemana, conversor.mesConvertido)
    pedidos['ano_pedido'] = pedidos['ano_pedido'].replace(anoSemana, conversor.anoConvertido)
    
    pedidos['ano_sem_cobranca_data'] = pedidos['ano_sem_cobranca_data'].replace(anoSemana, conversor.dataConvertida)
    pedidos['ano_mes_cobranca'] = pedidos['ano_mes_cobranca'].replace(anoSemana, conversor.mesConvertido)
    pedidos['ano_cobranca'] = pedidos['ano_cobranca'].replace(anoSemana, conversor.anoConvertido)

In [326]:
#pedidos.to_csv(f'data/pedidos.csv', sep= ';', index= False)

## Fechamentos

In [327]:
tabela = 'FECHAMENTO'
fechamentos = datab.fetchAllSQL(f'SELECT * FROM {tabela}')
#indexItens = datab.fetchAllSQL(f"select rdb$field_name from rdb$relation_fields where rdb$relation_name='{tabela}'")
indexItens = ['id_pedido', 'status', 'ano_sem_pedido', 'ano_sem_cobranca', 'quant_devolvida', 'quant_vendida', 'valor_venda_pedido', 'per_comissao', 'valor_comissao_sacoleira', 'valor_liquido', 'valor_sem_comissao', 'valor_liquido_recebido', 'valor_recebido', 'valor_pepino', 'valor_fator_venda', 'valor_dinheiro', 'valor_cheque', 'valor_cartao', 'data_fechamento', 'hora_fechamento', 'quantidade_devolvida_original', 'valor_venda_original', 'data_conferencia', 'data_fechamento_vazia', 'status_original', 'data_original', 'hora_original', 'gps', 'gps_original', 'valor_boleto', 'valor_deposito', 'valor_credito', 'valor_debito', 'valor_vend_ped_com_fixa', 'valor_vend_liq_ped_com_fixa']
fechamentos = pd.DataFrame(fechamentos)
fechamentos.columns = (indexItens) # type: ignore

In [328]:
fechamentos = fechamentos.drop(columns= ['valor_sem_comissao', 'gps', 'gps_original', 'valor_boleto', 'valor_deposito', 'valor_credito', 'valor_debito', 'valor_vend_ped_com_fixa', 'valor_vend_liq_ped_com_fixa', 'data_conferencia', 'data_fechamento_vazia'])


In [329]:
fechamentos = pd.merge(fechamentos, pedidos[['id_vendedor', 'id_praca', 'id_pedido', 'valor_total_pedido', 'id_sacoleira']], on= 'id_pedido', how='left' )
fechamentos = pd.merge(fechamentos, pracas[['id_praca', 'descricao_praca']], on= 'id_praca', how= 'left')

fechamentos = fechamentos.dropna(subset=['id_pedido'])

In [330]:
todosAnoSem = pd.concat([fechamentos['ano_sem_pedido'], fechamentos['ano_sem_cobranca']]).unique() # type: ignore


In [331]:
todosAnoSem = pd.concat([fechamentos['ano_sem_pedido'], fechamentos['ano_sem_cobranca']]).unique() # type: ignore

fechamentos['ano_sem_pedido_data'] = fechamentos['ano_sem_pedido']
fechamentos['ano_mes_pedido'] = fechamentos['ano_sem_pedido']
fechamentos['ano_pedido'] = fechamentos['ano_sem_pedido']

fechamentos['ano_sem_cobranca_data'] = fechamentos['ano_sem_cobranca']
fechamentos['ano_mes_cobranca'] = fechamentos['ano_sem_cobranca']
fechamentos['ano_cobranca'] = fechamentos['ano_sem_cobranca']

for anoSemana in todosAnoSem:
    conversor = conversorSemanas(anoSemana)
    fechamentos['ano_sem_pedido_data'] = fechamentos['ano_sem_pedido_data'].replace(anoSemana, conversor.dataConvertida)
    fechamentos['ano_mes_pedido'] = fechamentos['ano_mes_pedido'].replace(anoSemana, conversor.mesConvertido)
    fechamentos['ano_pedido'] = fechamentos['ano_pedido'].replace(anoSemana, conversor.anoConvertido)
    
    fechamentos['ano_sem_cobranca_data'] = fechamentos['ano_sem_cobranca_data'].replace(anoSemana, conversor.dataConvertida)
    fechamentos['ano_mes_cobranca'] = fechamentos['ano_mes_cobranca'].replace(anoSemana, conversor.mesConvertido)
    fechamentos['ano_cobranca'] = fechamentos['ano_cobranca'].replace(anoSemana, conversor.anoConvertido)

In [332]:
fechamentos = fechamentos.dropna()

In [333]:
fechamentos.to_csv('data/fechamentos.csv', sep= ';', index= False)

# PEDIDOS COMPLETO

In [334]:
pedidos['status'].unique()

array(['FECHADO', 'PEPINO', 'DIGITADO', 'TRANSFERIDO'], dtype=object)

In [335]:
pedidos = pedidos.merge(fechamentos[['id_pedido', 'quant_vendida', 'valor_venda_pedido', 'valor_liquido']], on='id_pedido', how='left')

In [336]:
pedidos['status'].unique()

array(['FECHADO', 'PEPINO', 'DIGITADO', 'TRANSFERIDO'], dtype=object)

In [337]:
pedidos.status.unique()

array(['FECHADO', 'PEPINO', 'DIGITADO', 'TRANSFERIDO'], dtype=object)

In [338]:
pedidos.to_csv(f'data/pedidos.csv', sep= ';', index= False)

In [ ]:
pd.set_option("display.max_columns", None)

## Pedidos Pecas

In [ ]:
tabela = 'PEDIDOSARTIGOS'
artigosPedido = datab.fetchAllSQL(f'SELECT * FROM {tabela}')
# indexItens = datab.fetchAllSQL(f"select rdb$field_name from rdb$relation_fields where rdb$relation_name='{tabela}'")
indexItens = ['id_pedido', 'ref', 'centro_estoque', 'quantidade_entregue','quantidade_devolvida', 'quantidade_vendida', 'quantidade_divergente', 'valor_unitario', 'valor_entregue', 'valor_vendido', 'status', 'id_praca', 'ano_sem_pedido', 'ano_sem_cobranca', 'quantidade_inserida', 'data_troca', 'id_artigo_velho', 'quantidade_devolvida_original', 'comissao', 'saida_remarcada', 'comissao_fixa']
artigosPedido = pd.DataFrame(artigosPedido)
artigosPedido.columns = (indexItens) # type: ignore

In [ ]:
artigosPedido = artigosPedido.drop(columns= ['centro_estoque', 'quantidade_divergente', 'id_artigo_velho', 'quantidade_devolvida_original', 'saida_remarcada', 'comissao_fixa', 'quantidade_inserida', 'data_troca', 'comissao'])

In [ ]:
artigosPedido.drop(artigosPedido[artigosPedido['ref'] == 1].index, inplace=True)
artigosPedido.drop(artigosPedido[artigosPedido['ref'] == 1001].index, inplace=True)

In [ ]:
artigosPedido = pd.merge(artigosPedido, pedidos[['id_pedido', 'cidade', 'uf', 'latitude', 'longitude', 'id_vendedor', 'descricao_vendedor', 'descricao_praca']], on='id_pedido', how= 'left')
artigosPedido = artigosPedido.merge(artigos[['ref', 'artigo']], on='ref', how= 'left')


In [ ]:
todosAnoSem = pd.concat([artigosPedido['ano_sem_pedido'], artigosPedido['ano_sem_cobranca']]).unique() # type: ignore

artigosPedido['ano_sem_pedido_data'] = artigosPedido['ano_sem_pedido']
artigosPedido['ano_mes_pedido'] = artigosPedido['ano_sem_pedido']
artigosPedido['ano_pedido'] = artigosPedido['ano_sem_pedido']

artigosPedido['ano_sem_cobranca_data'] = artigosPedido['ano_sem_cobranca']
artigosPedido['ano_mes_cobranca'] = artigosPedido['ano_sem_cobranca']
artigosPedido['ano_cobranca'] = artigosPedido['ano_sem_cobranca']

for anoSemana in todosAnoSem:
    conversor = conversorSemanas(anoSemana)
    artigosPedido['ano_sem_pedido_data'] = artigosPedido['ano_sem_pedido_data'].replace(anoSemana, conversor.dataConvertida)
    artigosPedido['ano_mes_pedido'] = artigosPedido['ano_mes_pedido'].replace(anoSemana, conversor.mesConvertido)
    artigosPedido['ano_pedido'] = artigosPedido['ano_pedido'].replace(anoSemana, conversor.anoConvertido)
    
    artigosPedido['ano_sem_cobranca_data'] = artigosPedido['ano_sem_cobranca_data'].replace(anoSemana, conversor.dataConvertida)
    artigosPedido['ano_mes_cobranca'] = artigosPedido['ano_mes_cobranca'].replace(anoSemana, conversor.mesConvertido)
    artigosPedido['ano_cobranca'] = artigosPedido['ano_cobranca'].replace(anoSemana, conversor.anoConvertido)

In [ ]:
artigosPedido.to_csv(f'data/artigosPedido.csv', sep= ';', index= False)

## Desempenhos Praças

In [ ]:
tabela = 'DESEMPENHOPRACAS'
desempenhoPraca = datab.fetchAllSQL(f'SELECT * FROM {tabela}')
indexItens = datab.fetchAllSQL(f"select rdb$field_name from rdb$relation_fields where rdb$relation_name='{tabela}'")
indexItens = ['id_desempenhopraca', 'id_praca', 'ano_sem_cobranca', 'id_vendedor', 'valor_total_venda', 'pedidos_novos', 'pedidos_digitados', 'pedidos_fechados', 'pedidos_pepinos', 'nao_sei', 'total_bruto_cobranca', 'total_novos_pepinos', 'per_pepino_cobranca', 'valor_recebido', 'valor_pepino_recebidos', 'total_recebido', 'media_comissao_cliente', 'vl_total_pedidos_liquido', 'fator_cobranca', 'valor_comissao_vendedor', 'comissao_vendedor_valor', 'ajuda_de_custo', 'nao_sei_2', 'faturamento_liquido', 'dinheiro_entregue', 'deposito', 'nao_sei_3', 'cheque', 'nao_sei_5', 'cheques_sf_novos', 'valor_total_pepinos_abertos', 'valor_pepinos_cobranca_anterior', 'valor_total_cheques_abertos', 'valor_cheques_cobranca_anterior', 'pedidos_novos_pep', 'codigo_pep_ab', 'pedidos_novos_ch_ab', 'codigo_cheques_ab', 'nao_sei_10', 'cliente_abaixo_meta', 'cod_clientes_abaixo_meta', 'clientes_novos', 'valor_medio_recebido', 'cartao', 'ano_sem_ped', 'nao_sei_6', 'valor_desconto', 'per_com_ven_sc', 'valor_com_vend_sc', 'per_com_vend_fix', 'valor_com_vend_rt_fix', 'clientes_venda_media_abaixo_meta', 'clientes_ped_novo_abaixo_meta', 'clientes_ped_novo_media_abaixo_meta', 'pedidos_transferidos_cobrados', 'notas_reais', 'per_comissao_cliente', 'valor_comissao_cliente', 'pd_venda', 'valor_total_pd_venda']
desempenhoPraca = pd.DataFrame(desempenhoPraca)
desempenhoPraca.columns = (indexItens)

In [ ]:
desempenhoPraca = pd.merge(desempenhoPraca, pracas[['id_praca','descricao_praca', 'status_praca']], on= 'id_praca')
desempenhoPraca = pd.merge(desempenhoPraca, vendedores[['id_vendedor', 'descricao_vendedor']], on= 'id_vendedor')


In [ ]:
desempenhoPraca['ano_sem_pedido_data'] = desempenhoPraca['ano_sem_ped'].apply(lambda x: pd.to_datetime(x+'/1', format= '%y/%W/%w'))
desempenhoPraca['ano_sem_cobranca_data'] = desempenhoPraca['ano_sem_cobranca'].apply(lambda x: pd.to_datetime(x+'/1', format= '%y/%W/%w'))
desempenhoPraca['ano_mes_pedido'] = desempenhoPraca['ano_sem_pedido_data'].dt.strftime('%Y-%m')
desempenhoPraca['ano_mes_cobranca'] = desempenhoPraca['ano_sem_cobranca_data'].dt.strftime('%Y-%m')
desempenhoPraca['ano_cobranca'] = desempenhoPraca['ano_sem_cobranca_data'].dt.strftime('%Y').astype(int)
desempenhoPraca['mes_cobranca'] = desempenhoPraca['ano_sem_cobranca_data'].dt.strftime('%m').astype(int)

In [ ]:
desempenhoPraca = desempenhoPraca.drop(columns= ['nao_sei', 'per_pepino_cobranca', 'nao_sei_2', 'dinheiro_entregue', 'deposito', 'nao_sei_3', 'cheque', 'nao_sei_5', 'cheques_sf_novos', 'valor_total_pepinos_abertos', 'valor_pepinos_cobranca_anterior', 'valor_total_cheques_abertos', 'valor_cheques_cobranca_anterior', 'pedidos_novos_pep', 'codigo_pep_ab', 'pedidos_novos_ch_ab', 'codigo_cheques_ab', 'nao_sei_10', 'cartao', 'nao_sei_6', 'valor_desconto', 'per_com_ven_sc', 'valor_com_vend_sc', 'per_com_vend_fix', 'valor_com_vend_rt_fix', 'pedidos_transferidos_cobrados', 'notas_reais', 'per_comissao_cliente', 'valor_comissao_cliente', 'pd_venda', 'valor_total_pd_venda'])  # type: ignore


In [ ]:
desempenhoPraca.to_csv('data/desempenhoPraca.csv', sep= ';', index= False)

## Outros